# LLMFS Examples

This notebook demonstrates the usage of LLMFS (LLM-powered Memory Filesystem), a FUSE-based memory filesystem backed by JSON that can generate filesystem structures using OpenAI's GPT models.

## Setup

First, let's import the necessary components and ensure we have our dependencies:

In [ ]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv
from llmfs.core.operations import Memory
from llmfs.content.generator import generate_filesystem

# Load environment variables from .env file
load_dotenv()

def generate_and_show(prompt):
    """Generate filesystem structure from prompt and show it"""
    try:
        # Generate filesystem structure
        print(f"Generating filesystem from prompt: {prompt}")
        fs_data = generate_filesystem(prompt)
        
        # Create filesystem instance
        fs = Memory(fs_data["data"])
        
        # Show structure
        print("\nGenerated filesystem structure:")
        def print_tree(path="/", indent=""):
            entries = fs.readdir(path, None)
            for entry in sorted(entries):
                if entry in [".", ".."]:
                    continue
                full_path = f"{path}/{entry}" if path != "/" else f"/{entry}"
                try:
                    # Check if it's a directory
                    fs.readdir(full_path, None)
                    print(f"{indent}├── {entry}/")
                    print_tree(full_path, indent + "│   ")
                except:
                    print(f"{indent}├── {entry}")
        
        print_tree()
        
        # Show filesystem data
        print("\nFilesystem data:")
        print(json.dumps(fs_data["data"], indent=2))
        
    except Exception as e:
        print(f"Error: {e}")

## Example 1: Python Project Structure

Let's create a Python project structure and see what gets generated:

In [ ]:
generate_and_show("Create a Python project with src directory, tests, documentation, and typical configuration files")

## Example 2: Web Development Project

Now let's generate a web development project structure:

In [ ]:
generate_and_show("Create a web development project with HTML, CSS, JavaScript files, and typical assets like images and fonts")

## Fun Retro OS Examples

Let's explore some nostalgic filesystem structures!

### Windows 95 System Directory

In [ ]:
generate_and_show("Create a Windows 95 system directory structure with Program Files, My Documents, and include classic games like Minesweeper and Solitaire")

### Commodore 64 Directory

In [ ]:
generate_and_show("Generate a C64 disk structure with BASIC programs, game files, and demo scene productions, including PRG files and SEQ data files")

### AmigaOS Workbench

In [ ]:
generate_and_show("Create an AmigaOS Workbench structure with the Utilities drawer, Shell commands, and classic demo files including .MOD music files")

### OpenBSD Development

In [ ]:
generate_and_show("Set up an OpenBSD development environment with ports tree structure, X11 configuration, and typical BSD daemon configuration files")

### MS-DOS Gaming

In [ ]:
generate_and_show("Create a DOS gaming directory with Commander Keen, Wolfenstein 3D, and typical DOS game files including .EXE, .COM, and .BAT files")

### Classic Mac OS

In [ ]:
generate_and_show("Generate a System 7 directory structure with Control Panels, Extensions, and classic Mac applications including HyperCard stacks")

### ZX Spectrum

In [ ]:
generate_and_show("Create a ZX Spectrum tape directory structure with game files, BASIC listings, and screen$ display files")

### BeOS Desktop

In [ ]:
generate_and_show("Set up a BeOS desktop environment with typical application bundles, Be filesystem attributes, and media files")